In [15]:
import onnxruntime as ort
import numpy as np
from keras_image_helper import create_preprocessor

onnx_model_path = "hair_classifier_v1.onnx"
session = ort.InferenceSession(onnx_model_path, providers=["CPUExecutionProvider"])

In [7]:
inputs = session.get_inputs()
outputs = session.get_outputs()

input_name = inputs[0].name
output_name = outputs[0].name

In [9]:
output_name

'output'

In [17]:
def preprocess_pytorch(X):
    # X: shape (1, 299, 299, 3), dtype=float32, values in [0, 255]
    X = X / 255.0

    mean = np.array([0.485, 0.456, 0.406]).reshape(1, 3, 1, 1)
    std = np.array([0.229, 0.224, 0.225]).reshape(1, 3, 1, 1)

    # Convert NHWC → NCHW
    # from (batch, height, width, channels) → (batch, channels, height, width)
    X = X.transpose(0, 3, 1, 2)

    # Normalize
    X = (X - mean) / std

    return X.astype(np.float32)


preprocessor = create_preprocessor(preprocess_pytorch, target_size=(200, 200))

In [20]:
img = preprocessor.from_url("https://habrastorage.org/webt/yf/_d/ok/yf_dokzqy3vcritme8ggnzqlvwa.jpeg")

In [21]:
img.shape

(1, 3, 200, 200)

In [23]:
img[0, 0, 0, 0]

np.float32(-1.0732939)

In [24]:
result = session.run([output_name], {input_name: img})

In [25]:
result

[array([[0.09156627]], dtype=float32)]